In [151]:
import numpy as np
import pandas as pd

import random
import string

import itertools
from itertools import permutations
from itertools import combinations

import math

In [2]:
from kmodes.kmodes import KModes

In [3]:
# data = {
#     'Greg':['no','Maudet','Dauphine'],
#     'Audrey':['no','Maudet','Dauphine'],
#     'Nabil':['yes','S','Finance'],
#     'Clélia':['yes','S','-'],
#     'Livie':['no','LivieJu','-'],
#     'Julien':['no','LivieJu','-'],
#     'Mag':['no','Martin','FamilleGael'],
#     'Fab':['no','Martin','FamilleGael'],
#     'Val':['no','Valex','FamilleGael'],
#     'Alex':['no','Valex','FamilleGael']
# }

In [4]:
# data = {
#     'Papa':['yes','yes','Savoure','-'],
#     'Maman':['yes','yes','Savoure','-'],
#     'Marion':['yes','yes','S','-'],
#     'Bruno':['yes','yes','S','-'],
# }

In [132]:
df = pd.read_csv('/Users/gaelsavoure/Downloads/Algo - amis_familles.csv', header=None)
poids = [int(x) for x in df.loc[0].values[2:]]
columns = list(df.loc[1].values[2:])
df = df.iloc[2:]
df['Nom'] = df[0]+df[1]
df = df.drop(columns=[0,1])
df = df.set_index('Nom')
df.index.name = None
df.columns = columns
df.head(3)

,Couple,Vieux,Groupe d'affinités 1,Groupe d'affinités 2
NabilLahmek,-,non,-,nab_aud_gre
FlorentBesse,-,non,scouts,-
XavierAuffray,xav_mar,non,scouts,-


In [152]:
print('Number of tables should be approximately {}'.format(math.ceil(df.shape[0]/10)))

Number of tables should be approximately 14


In [133]:
def generate_random_string(length=2):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

In [134]:
df_fill = df.copy()

In [135]:
# Create a set to store generated random strings
random_strings_set = set()

for column in df.columns:
    for idx, value in df[column].items():
        if value == "-":
            # Generate a random string that is not in the set
            new_random_string = generate_random_string()
            while new_random_string in random_strings_set:
                new_random_string = generate_random_string()

            # Replace the "-" cell with the new random string
            df_fill.at[idx, column] = new_random_string
            random_strings_set.add(new_random_string)

In [136]:
df_fill.head()

,Couple,Vieux,Groupe d'affinités 1,Groupe d'affinités 2
NabilLahmek,Yd,non,w2,nab_aud_gre
FlorentBesse,Ju,non,scouts,GB
XavierAuffray,xav_mar,non,scouts,zW
MarionAuffray,xav_mar,non,scouts,Ka
FrançoisPinget,2J,non,scouts,lf


In [137]:
def duplicate_column(df, s, n):
    if s not in df.columns:
        raise ValueError(f"Column '{s}' does not exist in the DataFrame.")
    
    new_df = df.copy()
    
    for i in range(1, n):
        new_column_name = f"{s}_{i+1}"
        new_df[new_column_name] = df[s]
    
    return new_df

In [138]:
df_poids = df_fill.copy()
for idx, column_name in enumerate(columns):
    df_poids = duplicate_column(df_poids, column_name, poids[idx])
df_poids.head()

,Couple,Vieux,Groupe d'affinités 1,Groupe d'affinités 2,Couple_2,Couple_3,Couple_4,Vieux_2,Vieux_3,Vieux_4,Groupe d'affinités 1_2
NabilLahmek,Yd,non,w2,nab_aud_gre,Yd,Yd,Yd,non,non,non,w2
FlorentBesse,Ju,non,scouts,GB,Ju,Ju,Ju,non,non,non,scouts
XavierAuffray,xav_mar,non,scouts,zW,xav_mar,xav_mar,xav_mar,non,non,non,scouts
MarionAuffray,xav_mar,non,scouts,Ka,xav_mar,xav_mar,xav_mar,non,non,non,scouts
FrançoisPinget,2J,non,scouts,lf,2J,2J,2J,non,non,non,scouts


In [12]:
# def group_cluster(cluster_sizes):
#     combinations_indexes = list(itertools.combinations(enumerate(cluster_sizes), 2))
#     for combination in combinations_indexes:
#         combination_indexes = [x[0] for x in combination]
#         combination_values = [x[1] for x in combination]
#         if np.sum(combination_values)==10:
#             cluster_sizes = [x for i, x in enumerate(cluster_sizes) if i not in combination_indexes]
#             cluster_sizes.append(10)
#             return cluster_sizes
#     return cluster_sizes

In [168]:
# clustering on big clusters
def divide_cluster(df_clusters, highest_cluster_name):
    n_clusters_start = len(df_clusters['cluster'].unique())
    n_clusters_list = [n_clusters_start+i for i in range(1,6)]
    df_clusters_divide = df_clusters.copy()
    for n_clusters in n_clusters_list:
        km = KModes(n_clusters=n_clusters, 
                    init='Huang',
                    n_init=1, 
                    verbose=False,
                    random_state=random_state)
        clusters = km.fit_predict(df_clusters_divide)
        cluster_sizes = df_clusters_divide.groupby('cluster').agg({'cluster':'count'})['cluster'].values
        df_clusters_divide['cluster'] = km.labels_
        if len(np.where(clusters_sizes>10)[0])>0:
            continue
        df_clusters_divide['clusters'] = df_clusters_divide['clusters']+np.max(highest_cluster_name)
        return df_clusters_divide
    print('Could not find smaller enough clusters')
    return df_clusters

In [139]:
# returns indexes of clusters that could be grouped because they sum to 10
def find_combinations(cluster_sizes, target_sum=10):
    result = []
    for r in range(1, len(cluster_sizes) + 1):
        combinations = list(itertools.combinations(enumerate(cluster_sizes), r))
        for combination in combinations:
            combination_indexes = [x[0] for x in combination]
            combination_values = [x[1] for x in combination]
            if sum(combination_values) == target_sum:
                if len(combination_values)==1:
                    continue
                result.append(combination_indexes)
    return result

In [140]:
# returns groups of clusters that have no cluster in common and that could be grouped together
def find_unique_combinations(list_of_list_of_clusters):
    result = []
    for r in range(1, len(list_of_list_of_clusters) + 1):
        for comb in combinations(list_of_list_of_clusters, r):
            combined = []
            for c in comb:
                combined.extend(c)
            # Check if the combined list has unique elements
            if len(set(combined)) == len(combined):
                result.append(list(comb))
    return result

In [141]:
# returns best combination of clusters i.e the one that groups the most clusters
def get_longest_list(list_of_list_of_clusters):
    return max(list_of_list_of_clusters, key=len)

In [142]:
# we give to all relevant clusters the same name as the first one from the group 
def group_clusters(df_clusters, best_combination):
    for cluster_group in best_combination:
        for cluster_name in cluster_group[1:]:
            df_clusters.loc[df_clusters['cluster']==cluster_name,'cluster'] = cluster_group[0]
    return df_clusters

In [159]:
df_clusters = df_poids.copy()
n_run = 30
n_clusters = 18 #len(np.unique(df_clusters['Groupe d\'affinités 1']))
found = False
df_clusters_best = None
best_sizes = None
number_full_tables_opt = 0
for random_state in np.arange(n_run):

    km = KModes(n_clusters=n_clusters, 
                init='Huang',
                n_init=1, 
                verbose=False,
                random_state=random_state)
    clusters = km.fit_predict(df_clusters)
    #     print(km.cluster_centroids_) # person that "represents" the cluster
    df_clusters['cluster'] = km.labels_
    cluster_names = list(df_clusters.groupby('cluster').agg({'cluster':'count'})['cluster'].index)
    cluster_sizes = df_clusters.groupby('cluster').agg({'cluster':'count'})['cluster'].values
    if len([size for size in cluster_sizes if size > 10])!=0:
        continue;

    print('Found clusters with small sizes only...')
    print(cluster_sizes)
    
    print('Finding all combinations that sum to 10...')
    all_combinations_10 = find_combinations(cluster_sizes)
    print('Finding all distinct combinations...')
    all_combinations_10_unique = find_unique_combinations(all_combinations_10)
    print('Finding the best combination...')
    best_combination_indexes = get_longest_list(all_combinations_10_unique)
    best_combination_names = best_combination_indexes.copy()
    df_clusters_grouped_1 = group_clusters(df_clusters, best_combination_names)
    cluster_sizes_grouped_1 = df_clusters.groupby('cluster').agg({'cluster':'count'})['cluster'].values
    
    cluster_names_2 = list(df_clusters_grouped_1.groupby('cluster').agg({'cluster':'count'})['cluster'].index)
    cluster_sizes_2 = df_clusters_grouped_1.groupby('cluster').agg({'cluster':'count'})['cluster'].values
    print('Finding all combinations that sum to 9...')
    all_combinations_9 = find_combinations(cluster_sizes_2, target_sum=9)
    print('Finding all distinct combinations...')
    all_combinations_9_unique = find_unique_combinations(all_combinations_9)
    print('Finding the best combination...')
    best_combination_indexes = get_longest_list(all_combinations_9_unique)
    best_combination_names = [[cluster_names_2[idx] for idx in combo_idx] for combo_idx in best_combination_indexes]
    df_clusters_grouped_2 = group_clusters(df_clusters_grouped_1, best_combination_names)
    cluster_sizes_grouped_2 = df_clusters_grouped_2.groupby('cluster').agg({'cluster':'count'})['cluster'].values
    
    number_full_tables10 = len(np.where(cluster_sizes_groupped==10)[0])
    number_full_tables9 = len(np.where(cluster_sizes_groupped==9)[0])
    number_full_tables = number_full_tables10 + number_full_tables9
    
    if number_full_tables>number_full_tables_opt:
        number_full_tables_opt = number_full_tables
        best_sizes = cluster_sizes_grouped_2.copy()
        df_clusters_best = df_clusters_grouped_2.copy()
#     if np.min(cluster_sizes_grouped)>8 and np.max(cluster_sizes_grouped)<12:
#         print('Good clusters found')
#         found = True
#         break
print()
# if not found:
#     print('No good cluster found')
print('best cluster sizes: {}'.format(best_sizes))


best cluster sizes: None


In [111]:
df_clusters_best_adjust = group_clusters(df_clusters_best, [[6,3]]).copy()

In [112]:
for idx, cluster_name in enumerate(df_clusters_best_adjust['cluster'].unique()):
    print('Table {}: {} personnes'.format(idx+1, df_clusters_best_adjust[df_clusters_best_adjust['cluster']==cluster_name].shape[0]))

Table 1: 10 personnes
Table 2: 10 personnes
Table 3: 10 personnes
Table 4: 11 personnes
Table 5: 10 personnes


--> we don't want someone from a group to be alone (e.g. a couple)

--> perform new clustering??

--> maybe easier to look for more clusters and group them by 10 as much as possible (easier than separating them...)

--> maybe help the algo to draw small clusters with another column of affinities

--> and simulate on random_state and on n_clusters...


The initial number of clusters should be lower than the number of groups of affinities (group 1); otherwise there's no real point using Kmodes...

TODO: 

- if cluster size > 10: recluster until you have size max=10 DONE?
- combine groups: don't mix old/young family/friends

In [180]:
import plotly.graph_objects as go

# Data for the bar chart
categories = ['Category A', 'Category B']
values = [0.3, 0.4]
error_min = [0.1, 0.2]
error_max = [0.7, 0.4]

# Create the bar trace with error bars
trace = go.Bar(
    x=categories,
    y=values,
    error_y=dict(
        type='data',  # Set error type to data
        symmetric=False,  # Asymmetric error bars
        array=error_max,  # Positive error values
        arrayminus=error_min  # Negative error values
    )
)

# Layout configuration
layout = go.Layout(
    title='Bar Chart with Custom Error Intervals',
    yaxis=dict(title='Value'),
    xaxis=dict(title='Category')
)

# Create the figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()
